In [156]:
import sys
import tqdm

sys.path.append("..")
from util import load_base_data

import biobss
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import neurokit2

SAMPLING_RATE = 300

In [2]:
X_train, y_train, X_test = load_base_data()

In [184]:
def mean_signal(signal):
    # cut the signal to the length of the shortest signal
    signal = np.array(signal)
    min_length = min([len(s) for s in signal])
    mean_length = np.mean([len(s) for s in signal])
    print("Mean signal length: ", mean_length, "min length: ", min_length)
    signal = [s[:min_length] for s in signal]
    return np.mean(signal, axis=0)


mean_train_ecg = mean_signal(X_train)

Mean signal length:  17807.0 min length:  17807


In [160]:
def get_loc_peaks(ecg):
    # filtered_ecg=biobss.preprocess.filter_signal(sig, sampling_rate=SAMPLING_RATE, signal_type='ECG', method='pantompkins')
    locs_peaks = biobss.ecgtools.ecg_detectpeaks(ecg, SAMPLING_RATE, "pantompkins")
    # peaks = ecg[locs_peaks]
    # info = biobss.sqatools.check_phys(locs_peaks, SAMPLING_RATE)
    return locs_peaks

In [48]:
def check_difference_to_mean(ecg, mean_ecg):
    loc_peaks = get_loc_peaks(mean_ecg)
    info = biobss.sqatools.template_matching(ecg, loc_peaks)
    return info

In [130]:
def calculate_features_based_on_difference_to_mean(ecg, mean_ecg):
    info = check_difference_to_mean(ecg, mean_ecg)
    # compute the mean, median and std of the correlation coefficients of the peaks compared to the mean signal
    mean = np.mean(info[0])
    median = np.median(info[0])
    std = np.std(info[0])
    return mean, median, std

In [53]:
def get_fiducials(ecg, loc_peaks):
    _, fiducials = neurokit2.ecg_delineate(
        ecg_cleaned=ecg, rpeaks=loc_peaks, sampling_rate=SAMPLING_RATE, method="peak"
    )

    # p_peaks_locs = fiducials["ECG_P_Peaks"]
    # q_peaks_locs = fiducials["ECG_Q_Peaks"]
    # s_peaks_locs = fiducials["ECG_S_Peaks"]
    # t_peaks_locs = fiducials["ECG_T_Peaks"]
    # p_onset_locs = fiducials["ECG_P_Onsets"]
    # t_offset_locs = fiducials["ECG_T_Offsets"]
    return fiducials

In [68]:
def compute_features_rpeaks(ecg):
    loc_peaks = get_loc_peaks(ecg)
    features_rpeaks = biobss.ecgtools.ecg_features.from_Rpeaks(
        ecg, loc_peaks, SAMPLING_RATE, average=False  # take the mean outside
    )
    return features_rpeaks

In [69]:
def compute_features_from_P_Q_R_S_T(ecg):
    loc_peaks = get_loc_peaks(ecg)
    fiducials = get_fiducials(ecg, loc_peaks)
    features = biobss.ecgtools.ecg_features.from_waves(
        ecg, loc_peaks, fiducials, SAMPLING_RATE, average=False  # Take the mean outside
    )
    return features

In [131]:
features_based_on_difference_to_mean_test = (
    calculate_features_based_on_difference_to_mean(X_train.iloc[0], mean_ecg)
)
features_rpeaks_test = compute_features_rpeaks(X_train.iloc[0])
features_from_P_Q_R_S_T_test = compute_features_from_P_Q_R_S_T(X_train.iloc[0])

In [150]:
def average_result(result):
    """input is a dict of dicts. Average over all the not nan values
       an example is
       1: {'ecg_a_R': -147.0,
     'ecg_RR0': 0.36666666666666664,
     'ecg_RR1': 0.84,
     'ecg_RR2': 0.7766666666666666,
     'ecg_RRm': 0.6611111111111111,
     'ecg_RR_0_1': 0.4365079365079365,
     'ecg_RR_2_1': 0.9246031746031745,
     'ecg_RR_m_1': 0.7870370370370371},
    2: {'ecg_a_R': -69.0,
     'ecg_RR0': 0.84,
     'ecg_RR1': 0.7766666666666666,
     'ecg_RR2': 0.89,
     'ecg_RRm': 0.8355555555555556,
     'ecg_RR_0_1': 1.0815450643776825,
     'ecg_RR_2_1': 1.145922746781116,
     'ecg_RR_m_1': 1.0758226037195995},"""
    number_of_results = sorted(list(result.keys()))

    if len(number_of_results) <= 0:
        print("No results to average over")
        print(result)
        return [], []
    # get all keys
    keys = list(result[number_of_results[0]].keys())
    # create a dict with empty lists
    averaged_result = {key: [] for key in keys}
    std_result = {key: [] for key in keys}
    # iterate over all the results
    for index in number_of_results:
        r = result[index]
        # iterate over all the keys
        for key in keys:
            # if the value is not nan
            if not np.isnan(r[key]):
                # append it to the list
                averaged_result[key].append(r[key])
    # compute the mean of all the values
    for key in keys:
        std_result[key] = np.std(averaged_result[key])
        averaged_result[key] = np.mean(averaged_result[key])

    # sort the keys alphabetically and return a list of the values and a list of the keys
    sorted_keys = sorted(averaged_result.keys())

    # rename key to mean_key and std_key
    return_keys = []
    for key in sorted_keys:
        return_keys.append(key + "_mean")
        return_keys.append(key + "_std")

    return_values = []
    for key in sorted_keys:
        return_values.append(averaged_result[key])
        return_values.append(std_result[key])
    return return_values, return_keys

In [144]:
average_result(features_rpeaks_test)

No results to average over
{1: {'ecg_a_R': -147.0, 'ecg_RR0': 0.36666666666666664, 'ecg_RR1': 0.84, 'ecg_RR2': 0.7766666666666666, 'ecg_RRm': 0.6611111111111111, 'ecg_RR_0_1': 0.4365079365079365, 'ecg_RR_2_1': 0.9246031746031745, 'ecg_RR_m_1': 0.7870370370370371}, 2: {'ecg_a_R': -69.0, 'ecg_RR0': 0.84, 'ecg_RR1': 0.7766666666666666, 'ecg_RR2': 0.89, 'ecg_RRm': 0.8355555555555556, 'ecg_RR_0_1': 1.0815450643776825, 'ecg_RR_2_1': 1.145922746781116, 'ecg_RR_m_1': 1.0758226037195995}, 3: {'ecg_a_R': 399.0, 'ecg_RR0': 0.7766666666666666, 'ecg_RR1': 0.89, 'ecg_RR2': 0.8533333333333334, 'ecg_RRm': 0.84, 'ecg_RR_0_1': 0.8726591760299625, 'ecg_RR_2_1': 0.9588014981273408, 'ecg_RR_m_1': 0.9438202247191011}, 4: {'ecg_a_R': -46.0, 'ecg_RR0': 0.89, 'ecg_RR1': 0.8533333333333334, 'ecg_RR2': 0.8433333333333334, 'ecg_RRm': 0.8622222222222223, 'ecg_RR_0_1': 1.04296875, 'ecg_RR_2_1': 0.98828125, 'ecg_RR_m_1': 1.0104166666666667}, 5: {'ecg_a_R': -80.0, 'ecg_RR0': 0.8533333333333334, 'ecg_RR1': 0.843333333

([0.8164583333333333,
  0.06360106513346385,
  0.8236458333333333,
  0.028877848284243373,
  0.8232291666666667,
  0.028831218311156325,
  0.9921928659291079,
  0.07982498319270012,
  1.0002605372265776,
  0.03958271714975402,
  0.9974844677185619,
  0.034642660812381286,
  0.8211111111111111,
  0.0299607510743351,
  -22.984375,
  143.15054271940213],
 ['ecg_RR0_mean',
  'ecg_RR0_std',
  'ecg_RR1_mean',
  'ecg_RR1_std',
  'ecg_RR2_mean',
  'ecg_RR2_std',
  'ecg_RR_0_1_mean',
  'ecg_RR_0_1_std',
  'ecg_RR_2_1_mean',
  'ecg_RR_2_1_std',
  'ecg_RR_m_1_mean',
  'ecg_RR_m_1_std',
  'ecg_RRm_mean',
  'ecg_RRm_std',
  'ecg_a_R_mean',
  'ecg_a_R_std'])

In [185]:
def make_all_features(ecg_pandas, mean_train_ecg):
    ecg_array = ecg_pandas.to_numpy()
    print(ecg_array[0].shape)

    feature_names = [
        "Mean correlation to the average signal",
        "Median correlation to the average signal",
        "Std correlation to the average signal",
    ]
    feature_names = (
        feature_names + average_result(compute_features_rpeaks(ecg_array[0]))[1]
    )
    feature_names = (
        feature_names + average_result(compute_features_from_P_Q_R_S_T(ecg_array[0]))[1]
    )

    num_features_for_rpeaks = len(
        average_result(compute_features_rpeaks(ecg_array[0]))[1]
    )
    num_features_for_P_Q_R_S_T = len(
        average_result(compute_features_from_P_Q_R_S_T(ecg_array[0]))[1]
    )

    mean_ecg = mean_train_ecg
    features_based_on_difference_to_mean = []
    features_rpeaks = []
    features_from_P_Q_R_S_T = []
    droped_indices = []
    for index, ecg in tqdm.tqdm(enumerate(ecg_array)):
        features_based_on_difference_to_mean.append(
            calculate_features_based_on_difference_to_mean(ecg, mean_ecg)
        )
        interim_features_rpeaks = average_result(compute_features_rpeaks(ecg))
        if len(interim_features_rpeaks[1]) != num_features_for_rpeaks:
            print(
                f"ERROR: Number of features for rpeaks is not consistent, Droping index {index}"
            )
            droped_indices.append(index)
            features_based_on_difference_to_mean.pop()
        else:
            features_rpeaks.append(interim_features_rpeaks[0])

            interim_features_from_P_Q_R_S_T = average_result(
                compute_features_from_P_Q_R_S_T(ecg)
            )
            if len(interim_features_from_P_Q_R_S_T[1]) != num_features_for_P_Q_R_S_T:
                print(
                    f"ERROR: Number of features for P_Q_R_S_T is not consistent, Droping index {index}"
                )
                droped_indices.append(index)
            else:
                features_from_P_Q_R_S_T.append(interim_features_from_P_Q_R_S_T[0])

    # make np array
    features_based_on_difference_to_mean = np.array(
        features_based_on_difference_to_mean
    )
    features_rpeaks = np.array(features_rpeaks)
    features_from_P_Q_R_S_T = np.array(features_from_P_Q_R_S_T)

    return (
        np.concatenate(
            (
                features_based_on_difference_to_mean,
                features_rpeaks,
                features_from_P_Q_R_S_T,
            ),
            axis=1,
        ),
        feature_names,
        droped_indices,
    )

In [186]:
make_all_features(X_train.iloc[0:2], mean_train_ecg)

(17807,)


2it [00:00,  6.18it/s]


(array([[ 2.72666876e-01,  2.78979461e-01,  3.24436380e-01,
          8.16458333e-01,  6.36010651e-02,  8.23645833e-01,
          2.88778483e-02,  8.23229167e-01,  2.88312183e-02,
          9.92192866e-01,  7.98249832e-02,  1.00026054e+00,
          3.95827171e-02,  9.97484468e-01,  3.46426608e-02,
          8.21111111e-01,  2.99607511e-02, -2.29843750e+01,
          1.43150543e+02, -1.50303030e+02,  1.36208727e+02,
          9.04253571e-01,  5.77364717e-01,  1.90349863e+00,
          2.92669222e+01,  1.29132402e+01,  8.13911554e+01,
         -1.23619045e-01,  3.34376035e+00,  2.49982043e+00,
          7.72055974e+00, -1.70838710e+02,  1.49281850e+02,
          1.32644068e+02,  1.58040304e+02,  5.08636364e+01,
          1.46347230e+02, -1.66935484e+01,  4.26677293e+01,
          2.82152542e+02,  8.50441189e+01, -6.75555556e+01,
          1.41062046e+02,  1.03567758e+00,  8.38101046e+00,
          3.75615526e-01,  8.74806200e+00, -2.19690892e-01,
          5.34801962e-01,  2.98033333e+0

# Make the features

In [164]:
features_X_train, feature_names, droped_indices = make_all_features(
    X_train, mean_train_ecg
)

(17807,)
Mean signal length:  17807.0 min length:  17807


75it [00:09,  7.81it/s]/Users/ericschreiber/miniconda3/envs/ml/lib/python3.10/site-packages/numpy/core/_methods.py:264: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/ericschreiber/miniconda3/envs/ml/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
85it [00:10,  8.91it/s]

No results to average over
{}
ERROR: Number of features for rpeaks is not consistent, Droping index 83


105it [00:13,  6.97it/s]/Users/ericschreiber/miniconda3/envs/ml/lib/python3.10/site-packages/numpy/core/_methods.py:264: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/ericschreiber/miniconda3/envs/ml/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
432it [00:52, 12.00it/s]

No results to average over
{}
ERROR: Number of features for rpeaks is not consistent, Droping index 429


...

1046it [02:07, 11.25it/s]/Users/ericschreiber/miniconda3/envs/ml/lib/python3.10/site-packages/numpy/core/_methods.py:264: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/ericschreiber/miniconda3/envs/ml/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
1351it [02:46, 10.68it/s]/Users/ericschreiber/miniconda3/envs/ml/lib/python3.10/site-packages/numpy/core/_methods.py:264: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/ericschreiber/miniconda3/envs/ml/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
1870it [03:51,  6.98it/s]/Users/ericschreiber/miniconda3/envs/ml/lib/python3.10/site-packages/numpy/core/_methods.py:264: RuntimeWarning: Degrees of freedom <

No results to average over
{}
ERROR: Number of features for rpeaks is not consistent, Droping index 3424


3667it [07:34,  7.95it/s]/Users/ericschreiber/miniconda3/envs/ml/lib/python3.10/site-packages/numpy/core/_methods.py:264: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/ericschreiber/miniconda3/envs/ml/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
4023it [08:17,  8.67it/s]/Users/ericschreiber/miniconda3/envs/ml/lib/python3.10/site-packages/numpy/core/_methods.py:264: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/ericschreiber/miniconda3/envs/ml/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
4165it [08:34,  8.52it/s]

No results to average over
{}
ERROR: Number of features for rpeaks is not consistent, Droping index 4163


5117it [10:31,  8.10it/s]


In [173]:
features_X_test.shape
# feature_names
# droped_indices

(5112, 81)

# Check the data for inf and nan

In [207]:
# How many infs
print(f"Number of infs: {np.sum(np.isinf(features_X_train))}")
# Which columns have infs
cols_with_infs = np.where(np.isinf(features_X_train))[1]
cols_with_infs_unique = np.unique(cols_with_infs)
print(f"Columns with infs: {len(cols_with_infs_unique)}")
# Numbers bigger than 10000
print(f"Number of values bigger than 10000: {np.sum(features_X_train > 10000)}")
# Cap everything bigger than 10000 to 10000
features_X_train[features_X_train > 10000] = 10000
# Numbers smaller than -10000
print(f"Number of values smaller than -10000: {np.sum(features_X_train < -10000)}")
# Cap everything smaller than -10000 to -10000
features_X_train[features_X_train < -10000] = -10000
# Cap inf to 20000
features_X_train[np.isinf(features_X_train)] = 20000
# Cap -inf to -20000
features_X_train[np.isneginf(features_X_train)] = -20000

# How many nans
print(f"Number of nans: {np.sum(np.isnan(features_X_train))}")
# Which columns have nans
cols_with_nans = np.where(np.isnan(features_X_train))[1]
cols_with_nans_unique = np.unique(cols_with_nans)
print(f"Columns with nans: {len(cols_with_nans_unique)}")

Number of infs: 2842
Columns with infs: 13
Number of values bigger than 10000: 840
Number of values smaller than -10000: 2002
Number of nans: 3210
Columns with nans: 58


In [199]:
y_train_droped = y_train.drop(droped_indices)
# reset the index
y_train_droped = y_train_droped.reset_index(drop=True)
# name the index column id
y_train_droped.index.name = "id"

In [208]:
# Write the features to a csv file
df = pd.DataFrame(features_X_train, columns=feature_names)
X_train_save_path = "/Users/ericschreiber/dev/ETH/AML/Project_1/aml-2023/task2/data/feature_extraction/bioss_X_train.csv"
df.index.name = "id"
df.to_csv(X_train_save_path, index=True)
y_train_save_path = "/Users/ericschreiber/dev/ETH/AML/Project_1/aml-2023/task2/data/feature_extraction/bioss_y_train.csv"
y_train_droped.to_csv(y_train_save_path, index=True)

In [187]:
features_X_test, feature_names_test, droped_indices_test = make_all_features(
    X_test, mean_train_ecg
)

(17807,)


402it [00:49,  7.29it/s]/Users/ericschreiber/miniconda3/envs/ml/lib/python3.10/site-packages/numpy/core/_methods.py:264: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/ericschreiber/miniconda3/envs/ml/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
640it [01:18,  8.56it/s]/Users/ericschreiber/miniconda3/envs/ml/lib/python3.10/site-packages/numpy/core/_methods.py:264: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/ericschreiber/miniconda3/envs/ml/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
741it [01:31,  8.28it/s]/Users/ericschreiber/miniconda3/envs/ml/lib/python3.10/site-packages/numpy/core/_methods.py:264: RuntimeWarning: Degrees of freedom <= 0

No results to average over
{}
ERROR: Number of features for rpeaks is not consistent, Droping index 747


819it [01:40,  9.34it/s]/Users/ericschreiber/miniconda3/envs/ml/lib/python3.10/site-packages/numpy/core/_methods.py:264: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/ericschreiber/miniconda3/envs/ml/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
1186it [02:26,  6.29it/s]/Users/ericschreiber/miniconda3/envs/ml/lib/python3.10/site-packages/numpy/core/_methods.py:264: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/ericschreiber/miniconda3/envs/ml/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
1685it [03:29,  8.63it/s]/Users/ericschreiber/miniconda3/envs/ml/lib/python3.10/site-packages/numpy/core/_methods.py:264: RuntimeWarning: Degrees of freedom <=

No results to average over
{}
ERROR: Number of features for rpeaks is not consistent, Droping index 2032


2327it [04:50,  7.72it/s]

No results to average over
{}
ERROR: Number of features for rpeaks is not consistent, Droping index 2327


2392it [04:59,  7.84it/s]/Users/ericschreiber/miniconda3/envs/ml/lib/python3.10/site-packages/numpy/core/_methods.py:264: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/ericschreiber/miniconda3/envs/ml/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
2708it [05:40,  7.66it/s]/Users/ericschreiber/miniconda3/envs/ml/lib/python3.10/site-packages/numpy/core/_methods.py:264: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/ericschreiber/miniconda3/envs/ml/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
3310it [06:54,  8.28it/s]/Users/ericschreiber/miniconda3/envs/ml/lib/python3.10/site-packages/numpy/core/_methods.py:264: RuntimeWarning: Degrees of freedom <

In [189]:
features_X_test_double = features_X_test

In [190]:
# We need to add the dropped indices to the test indices. We fill those rows with zeros

droped_indices_test.sort(
    reverse=True
)  # we need to fill the indices from the back otherwise the indices will be wrong
for index in droped_indices_test:
    features_X_test = np.insert(features_X_test, index, 0, axis=0)

In [209]:
# How many infs
print(f"Number of infs: {np.sum(np.isinf(features_X_test))}")
# Which columns have infs
cols_with_infs = np.where(np.isinf(features_X_test))[1]
cols_with_infs_unique = np.unique(cols_with_infs)
print(f"Columns with infs: {len(cols_with_infs_unique)}")
# Numbers bigger than 10000
print(f"Number of values bigger than 10000: {np.sum(features_X_test > 10000)}")
# Cap everything bigger than 10000 to 10000
features_X_test[features_X_test > 10000] = 10000
# Numbers smaller than -10000
print(f"Number of values smaller than -10000: {np.sum(features_X_test < -10000)}")
# Cap everything smaller than -10000 to -10000
features_X_test[features_X_test < -10000] = -10000
# Cap inf to 20000
features_X_test[np.isinf(features_X_test)] = 20000
# Cap -inf to -20000
features_X_test[np.isneginf(features_X_test)] = -20000

Number of infs: 2002
Columns with infs: 13
Number of values bigger than 10000: 601
Number of values smaller than -10000: 1401


In [210]:
df_test = pd.DataFrame(features_X_test, columns=feature_names_test)
df_test.index.name = "id"
X_test_save_path = "/Users/ericschreiber/dev/ETH/AML/Project_1/aml-2023/task2/data/feature_extraction/bioss_X_test.csv"
df_test.to_csv(X_test_save_path, index=True)

In [192]:
print(features_X_test.shape)
print(features_X_test_double.shape)
print(X_test.shape)

(3411, 81)
(3408, 81)
(3411, 17807)
